###Notebook for creating test data and data-loading script for MongoDB

In [2]:
%%writefile test_data.txt
9,42.97456,119.332025,492,2008-08-20,12:09:04,walk
9,39.974658,106.332114,492,2008-08-20,12:09:07,bus
18,39.974893,136.331959,467,2008-08-20,12:10:35,NA
18,39.974903,126.331965,466,2008-08-20,12:10:37,subway
18,31.97492,136.331962,466,2008-08-20,12:10:39,subway
120,32.974927,116.331959,466,2008-08-20,12:10:41,walk
120,40.974928,116.331944,466,2008-08-20,12:10:43,walk
120,42.974933,120.33194,465,2008-08-20,12:10:45,bus
120,44.974927,116.331928,465,2008-08-20,12:10:47,subway
120,45.974927,116.331927,465,2008-08-20,12:10:49,walk
9,40.97456,106.332025,492,2008-08-20,12:09:04,walk
9,39.974658,106.332114,492,2008-08-20,12:09:07,bus
18,39.974893,108.331959,467,2008-08-20,12:10:35,bus
18,39.974903,121.331965,466,2008-08-20,12:10:37,walk
18,39.97492,106.331962,466,2008-08-20,12:10:39,bus
120,39.974927,112.331959,466,2008-08-20,12:10:41,walk
120,39.974928,122.331944,466,2008-08-20,12:10:43,walk
120,42.974933,118.33194,465,2008-08-20,12:10:45,bus
120,44.974927,110.331928,465,2008-08-20,12:10:47,walk
120,45.974927,100.331927,465,2008-08-20,12:10:49,walk
9,41.97456,118.332025,492,2008-08-20,12:09:04,walk
9,39.974658,119.332114,492,2008-08-20,12:09:07,bus
18,39.974893,136.331959,467,2008-08-20,12:10:35,bus
18,39.974903,116.331965,466,2008-08-20,12:10:37,walk
18,34.97492,104.331962,466,2008-08-20,12:10:39,bus
120,33.974927,116.331959,466,2008-08-20,12:10:41,walk
120,38.974928,107.331944,466,2008-08-20,12:10:43,walk
120,44.974933,119.33194,465,2008-08-20,12:10:45,bus
120,30.974927,120.331928,465,2008-08-20,12:10:47,bus
120,25.974927,123.331927,465,2008-08-20,12:10:49,walk
121,25.974927,123.331927,465,2008-08-20,12:10:49,walk

Overwriting test_data.txt


In [7]:
!cat test_data.txt

9,42.97456,119.332025,492,2008-08-20,12:09:04,walk
9,39.974658,106.332114,492,2008-08-20,12:09:07,bus
18,39.974893,136.331959,467,2008-08-20,12:10:35,NA
18,39.974903,126.331965,466,2008-08-20,12:10:37,subway
18,31.97492,136.331962,466,2008-08-20,12:10:39,subway
120,32.974927,116.331959,466,2008-08-20,12:10:41,walk
120,40.974928,116.331944,466,2008-08-20,12:10:43,walk
120,42.974933,120.33194,465,2008-08-20,12:10:45,bus
120,44.974927,116.331928,465,2008-08-20,12:10:47,subway
120,45.974927,116.331927,465,2008-08-20,12:10:49,walk
9,40.97456,106.332025,492,2008-08-20,12:09:04,walk
9,39.974658,106.332114,492,2008-08-20,12:09:07,bus
18,39.974893,108.331959,467,2008-08-20,12:10:35,bus
18,39.974903,121.331965,466,2008-08-20,12:10:37,walk
18,39.97492,106.331962,466,2008-08-20,12:10:39,bus
120,39.974927,112.331959,466,2008-08-20,12:10:41,walk
120,39.974928,122.331944,466,2008-08-20,12:10:43,walk
120,42.974933,118.33194,465,2008-08-20,12:10:45,bus
120,44.974927,110.331928,465,2008-08-20,12:10:47,w

In [81]:
%%writefile load.py
import sys
import os
import json
import pymongo
import datetime, pytz
import re
from pymongo import MongoClient
# from datetime import datetime
# from dateutil import tz

class LoadData():

    client = MongoClient('localhost', 27017)
    db = client.w209project #create/set database name as "w209project"
    collection = db.locs    #create/set collection name as "locs"
    
    def removeData(self):
        self.collection.remove({})
    
    def load(self,data_file, num_data):
        #set limit on number of lines of data
        if num_data=="all": limit=1e15
        else: limit=int(num_data)
        count=0

        with open(data_file, 'r') as f:
            for line in f:
                if count>=limit: break
                
                user_id,latitude,longitude,altitude,date,time,transport = line.strip().split(',')
                user_id = int(user_id)
                latidude = float(latitude)
                longitude = float(longitude)
                altidude = float(altitude)

                #ensure it's china timezone to ensure correct timezone input
                local = pytz.timezone ("Asia/Shanghai")
                naive = datetime.datetime.strptime (date+" "+time, "%Y-%m-%d %H:%M:%S")
                local_dt = local.localize(naive, is_dst=None)
#                 utc_dt = local_dt.astimezone (pytz.utc)
#                 print latitude,longitude,altitude,date,time,transport
                
                location = {"user_id":user_id,
                            "latitude":latitude,
                            "longitude":longitude,
                            "altitude":altitude,
                            "date_time":local_dt,
                            "transport":transport,
                            "created_at":datetime.datetime.utcnow()}
                self.collection.insert_one(location)
            
                count+=1

            print '\n',self.collection.count(),'records loaded into MongoDB'
    
if __name__ == '__main__':
    l = LoadData()
    data_file = sys.argv[1]    #name of data file
    remove_data = sys.argv[2]  #if true, then remove existing data
    num_data = sys.argv[3]     #number of data points (ie. no. of lines of data)

    if remove_data=="T":
        l.removeData()
    l.load(data_file,num_data.lower())



Overwriting load.py


In [ ]:
pip install intervaltree

In [79]:
%%writefile importdata.sh
#!/bin/bash
FILES="../../Geolife Trajectories 1.3/data_csv"
for f in "$FILES"/*.csv
do
  echo "Processing ${f} data file..."
  python load.py "${f}" F all
done
# http://www.cyberciti.biz/faq/bash-loop-over-file/

Overwriting importdata.sh


In [82]:
!sh importdata.sh

Processing ../../Geolife Trajectories 1.3/data_csv/000.csv data file...

173870 records loaded into MongoDB
Processing ../../Geolife Trajectories 1.3/data_csv/001.csv data file...

282477 records loaded into MongoDB
Processing ../../Geolife Trajectories 1.3/data_csv/002.csv data file...

530694 records loaded into MongoDB
Processing ../../Geolife Trajectories 1.3/data_csv/003.csv data file...

1015920 records loaded into MongoDB
Processing ../../Geolife Trajectories 1.3/data_csv/004.csv data file...

1455317 records loaded into MongoDB
Processing ../../Geolife Trajectories 1.3/data_csv/005.csv data file...

1564363 records loaded into MongoDB
Processing ../../Geolife Trajectories 1.3/data_csv/006.csv data file...

1596193 records loaded into MongoDB
Processing ../../Geolife Trajectories 1.3/data_csv/007.csv data file...

1683410 records loaded into MongoDB
Processing ../../Geolife Trajectories 1.3/data_csv/008.csv data file...

1761320 records loaded into MongoDB
Processing ../../Geoli

In [6]:
!python load.py test_data.txt T all



!python load.py ../../Geolife Trajectories 1.3/output 000.csv F all
!python load.py 001.csv F all
!python load.py 000.csv F all
!python load.py 000.csv F all
!python load.py 000.csv F all








31 records loaded into MongoDB


###PLEASE IGNORE BELOW

In [4]:
%%writefile load.py
from boto.s3.connection import S3Connection
from boto.s3.bucket import Bucket
from boto.s3.key import Key
import sys
import os
import json
import pymongo
import datetime
import re
from pymongo import MongoClient
from datetime import datetime
from dateutil import tz

BUCKET_NAME = 'w205-price-comparison-tool'

class LoadData():

    db = None
    client = None
    bucket = None

    def connectToDatabase(self):
        # connect to database
        #client = MongoClient()
        self.client = MongoClient('localhost', 27017)
        # client = MongoClient('mongodb://localhost:27017/')
        self.db = self.client.w205project
        # conn = S3Connection('CHANGE_THIS_your_key_id','CHANGE_THIS_your_secret_access_key')
        conn = S3Connection()
        self.bucket = conn.get_bucket(BUCKET_NAME)

    # helper function to convert num to double-digit string (eg. 4 -> "04")
    def convertNum(self, num_int):
        num_string = str(num_int)
        if num_int < 10:
            num_string = "0"+num_string
        return num_string

    # helper function to convert string to boolean values
    def convertBoolean(self, string):
        if string == "T" or string == "True":
            return True
        if string == "F" or string == "False":
            return False

    # main function to load data into MongoDB
    def main(self,loc_c,prod_c,list_c,listU_c,mode,load_listing_via_s3,erase_listings,from_date_s="",to_date_s=""):

        if mode=='supervised':
            CLEAN_KEY_PREFIX = "clean/supervised/"
            keyname_data_start_index = 17  #used for determining scraped data below
            keyname_data_end_index = 27
            collectionUniqueListing = 'listing_u_sups'  #used for aggregation function output for unique listing
        elif mode == 'unsupervised':
            CLEAN_KEY_PREFIX = "clean/unsupervised/"
            keyname_data_start_index = 19
            keyname_data_end_index = 29
            collectionUniqueListing = 'listing_u_unsups'
        else:
            print "Incorrect mode entered. Please enter 'supervised' or 'unsupervised' only"
            return
    
        # format date-time (into UTC)
        if from_date_s == "":
            from_date = datetime.strptime('1900-01-01','%Y-%m-%d')
        else:
            from_date = datetime.strptime(from_date_s,'%Y-%m-%d')
    
        if to_date_s == "":
            to_date = datetime.utcnow()
        else:
            to_date = datetime.strptime(to_date_s,'%Y-%m-%d')


        # clear old entries
        prod_c.remove({})
        loc_c.remove({})
        listU_c.remove({})
        if erase_listings:
            list_c.remove({})
    
        if load_listing_via_s3:
            # get cleaned data key from S3
            print "Reading key name from S3"
            exp = re.compile(CLEAN_KEY_PREFIX)
            key_list = []
            for key in self.bucket.list():
                if exp.match(key.name):
                    scrape_date_s = key.name[keyname_data_start_index:keyname_data_end_index]
                    scrape_date = datetime.strptime(scrape_date_s, '%Y-%m-%d')
                    # print "from_date: ", from_date
                    # print "scrape_date: ", scrape_date
                    # print "to_date: ", to_date
                    # print "check from_date: ", scrape_date >= from_date
                    # print "check to_date: ", scrape_date <= to_date
                    if scrape_date >= from_date and scrape_date <= to_date:
                        key_list.append(key)

            # loop through each city json file
            print "\nStarting process of S3 -> MongoDB ("+mode+" data)"
            countKey = 0
            countListing = 0
            listings_mongo = []


            for key in key_list:
                json_string = key.get_contents_as_string()

            # for i in range(0,2):
            #     key = key_list[i]
            #     json_string = key_list[0].get_contents_as_string()

                listings = json.loads(json_string)

                #date on c3 chart is the scrape date (not date of posting creation)
                scrape_date_s = key.name[keyname_data_start_index:keyname_data_end_index]

                # enter each listing per city json file into MongoDB
                for listing in listings:
                    pred_class = listing['pred_class']
                    if pred_class != -1:
                        if pred_class == 0:
                            product = "iPhone (unclassified)"
                        else:
                            product = "iPhone "+pred_class
                        # print "mongo scrape date: ",scrape_date_s
                        listing_mongo = {
                            "city":listing['area'],
                            "product":product,
                            "price": float(listing['price']),
                            "title":listing['title'],
                            "url":listing['url'].strip('\\'),
                            "scraped_at":datetime.strptime(scrape_date_s,"%Y-%m-%d"),
                            "created_at":datetime.strptime(listing['create_date'][:10],"%Y-%m-%d"),
                            "c3Date":scrape_date_s
                        }

                        listings_mongo.append(listing_mongo)
                        countListing = countListing + 1

                        # list_c.insert(listings_mongo) #for testing only
                        # listings_mongo = [] #for testing only

                # print "key_list:" + str(len(key_list))
                # print "countKey:" + str(countKey)
                # print scrape_date_s
                # print "hello: "+str(listings_mongo)+" ... end hello"

                if len(listings_mongo)>10000:
                    print "10000 listings are obtained from S3, now inserting into MongoDB"
                    list_c.insert(listings_mongo)
                    listings_mongo = []
                elif countKey == len(key_list)-1:
                    print "reached the last key in S3, now inserting into MongoDB"
                    list_c.insert(listings_mongo)

                countKey = countKey + 1
                print "S3 -> MongDB progress ("+mode+"): "+ str(round(countKey/float(len(key_list))*100,1))+"%"
    
        # insert unique product into unique listing collection in MongoDB, keepying latest post date
        pipe = 	[
            {
                "$group" : {
                    "_id":"$url",  # _id is now the url for this unique listing
                    "product":{"$first":"$product"},
                    "title":{"$first":"$title"},
                    "price":{"$first":"$price"},
                    "city":{"$first":"$city"},
                    "created_at":{"$first":"$created_at"},
                    "url":{"$first":"$url"}
                }
            },
            {"$sort": {"created_at":-1}},
            {"$out" : collectionUniqueListing}
        ]
        list_c.aggregate(pipeline=pipe,allowDiskUse=True)

        # insert unique product into products collection in MongoDB
        print "\nFollowing products are being entered into database (" +mode+")"
        prod_c.insert({"product":"All iPhones"})
        pipe = 	[
            {
                "$group" : {
                    "_id":"$product",
                    "posts": {"$sum":1}
                }
            },
            {"$sort" : {"_id":1}}
        ]
        info_by_product = listU_c.aggregate(pipeline=pipe)
        productList = []
        for product in info_by_product["result"]:
            if product["_id"] not in productList:
                productList.append(product["_id"])
                prod_c.insert({"product":product["_id"]})
        print productList

        # insert unique location into locations collection in MongoDB
        print "\nFollowing products are being entered into database (" +mode+")"
        loc_c.insert({"location":"Throughout U.S."})
        pipe = 	[
            {
                "$group" : {
                    "_id":"$city",
                    "posts": {"$sum":1}
                }
            },
            {"$sort" : {"_id":1}}
        ]
        info_by_location = listU_c.aggregate(pipeline=pipe)
        locationList = []
        for location in info_by_location["result"]:
            if location["_id"] not in locationList:
                locationList.append(location["_id"])
                loc_c.insert({"location":location["_id"]})
        print locationList
        print "\n##############################################"

    def load(self, mode, new, erase, start, end):

        self.connectToDatabase()

        input_s3_listing = self.convertBoolean(new)
        input_erase_listing = self.convertBoolean(erase)

        if mode =="supervised" or mode =="both":
            self.main(self.db.loc_sups,self.db.prod_sups,self.db.listing_sups,self.db.listing_u_sups,
                "supervised",input_s3_listing,input_erase_listing,start,end)
        if mode =="unsupervised" or mode =="both":
            self.main(self.db.loc_unsups,self.db.prod_unsups,self.db.listing_unsups,self.db.listing_u_unsups,
                "unsupervised",input_s3_listing,input_erase_listing,start,end)
        
        print '\nSummary of data loaded in MongoDB for SUPERVISED mode:'
        print 'No. of products:         '+str(self.db.prod_sups.count())
        print 'No. of locations:        '+str(self.db.loc_sups.count())
        print 'No. of general listings: '+str(self.db.listing_sups.count())
        print 'No. of unique listings:  '+str(self.db.listing_u_sups.count())
    
        print '\nSummary of data loaded in MongoDB for UNSUPERVISED mode:'
        print 'No. of products:         '+str(self.db.prod_unsups.count())
        print 'No. of locations:        '+str(self.db.loc_unsups.count())
        print 'No. of general listings: '+str(self.db.listing_unsups.count())
        print 'No. of unique listings:  '+str(self.db.listing_u_unsups.count())+'\n'

if __name__ == '__main__':

    # Input without providing external argument
    # Command line sample:   python load.py
    # main(db.loc_unsups,db.prod_unsups,db.listing_unsups,db.listing_u_unsups,'unsupervised',True,True)
    # main(db.loc_sups,db.prod_sups,db.listing_sups,db.listing_u_sups,'supervised',True,True,'2015-04-15','2015-04-15')
    
    # Input without providing external argument
    # Command line sample:   python load.py 'supervised' True True '2015-04-15' '2015-04-17'
    # Input via running command line script
    l = LoadData()
    l.load(sys.argv[1], sys.argv[2], sys.argv[3], sys.argv[4], sys.argv[5])
    

Writing load.py
